In [13]:
import brightway2 as bw
import bw2regional as bwr
from bw2regional.geojson_export import *
from bw2analyzer import ContributionAnalysis

In [2]:
bw.projects.set_current("EXIOBASE regionalized case study")

In [3]:
bw.databases

Databases dictionary with 3 object(s):
	EXIOBASE 3.3.17 hybrid
	Oil seeds
	biosphere3

In [4]:
[x for x in bw.methods if x[0] == "LC-IMPACT"]

[('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'All'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'All'),
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'),
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Average'),
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'),
 ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'Certain'),
 ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'All')]

In [5]:
bwr.geocollections

Geocollections dictionary with 23 objects, including:
	cocnut
	ecoregions
	exiobase-rows
	exiobase-rows-ecoregions
	exiobase-rows-particulate-matter
	exiobase-rows-watersheds-eq-sw-all
	exiobase-rows-watersheds-eq-sw-certain
	exiobase-rows-watersheds-hh
	oilpalm
	particulate-matter
Use `list(this object)` to get the complete list.

In [4]:
sorted(bwr.extension_tables)

['xt-oilpalm-exiobase-rows-ecoregions',
 'xt-oilpalm-exiobase-rows-particulate-matter',
 'xt-oilpalm-exiobase-rows-watersheds-eq-sw-all',
 'xt-oilpalm-exiobase-rows-watersheds-eq-sw-certain',
 'xt-oilpalm-exiobase-rows-watersheds-hh',
 'xt-oilpalm-world-topo-ecoregions',
 'xt-oilpalm-world-topo-particulate-matter',
 'xt-oilpalm-world-topo-watersheds-eq-sw-all',
 'xt-oilpalm-world-topo-watersheds-eq-sw-certain',
 'xt-oilpalm-world-topo-watersheds-hh',
 'xt-peanut-exiobase-rows-ecoregions',
 'xt-peanut-exiobase-rows-particulate-matter',
 'xt-peanut-exiobase-rows-watersheds-eq-sw-all',
 'xt-peanut-exiobase-rows-watersheds-eq-sw-certain',
 'xt-peanut-exiobase-rows-watersheds-hh',
 'xt-peanut-world-topo-ecoregions',
 'xt-peanut-world-topo-particulate-matter',
 'xt-peanut-world-topo-watersheds-eq-sw-all',
 'xt-peanut-world-topo-watersheds-eq-sw-certain',
 'xt-peanut-world-topo-watersheds-hh',
 'xt-rapeseed-exiobase-rows-ecoregions',
 'xt-rapeseed-exiobase-rows-particulate-matter',
 'xt-rapes

In [3]:
palm = next(ds for ds in bw.Database("Oil seeds") if ds['name'] == 'Palm oil refinery [NBD palm oil]')
sunflower = next(ds for ds in bw.Database("Oil seeds") if ds['name'] == 'Sunflower oil refinery [Refined sunflower oil]')
rapeseed = next(ds for ds in bw.Database("Oil seeds") if ds['name'] == 'Rapeseed oil refinery [Refined rapeseed oil]')
peanut = next(ds for ds in bw.Database("Oil seeds") if ds['name'] == 'Peanut oil refinery [Refined peanut oil]')
soybean = next(ds for ds in bw.Database("Oil seeds") if ds['name'] == 'Soybean oil refinery [CSBO processing]')

In [6]:
for obj in (rapeseed, sunflower, peanut, palm, soybean):
    print(obj.key)

('Oil seeds', 'Rapeseed oil refinery [Refined rapeseed oil]-EU27')
('Oil seeds', 'Sunflower oil refinery [Refined sunflower oil]-UA')
('Oil seeds', 'Peanut oil refinery [Refined peanut oil]-IN')
('Oil seeds', 'Palm oil refinery [NBD palm oil]-MY')
('Oil seeds', 'Soybean oil refinery [CSBO processing]-BR')


In [6]:
lca = bw.LCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
)
lca.lci()
lca.lcia()
lca.score

1.7649844252156698e-13

In [4]:
lca = bwr.TwoSpatialScalesLCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
)
lca.lci()
lca.lcia()
lca.score

1.3433522086801278e-12

In [4]:
lca = bwr.ExtensionTablesLCA(
    {palm: 1},
    ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
    xtable='xt-oilpalm-world-topo-ecoregions',
)
lca.lci()
lca.lcia()
lca.score

1.4804974820069666e-12

In [11]:
as_ia_spatial_scale(lca, geocollection="ecoregions", filepath="land_use_palm_oil")

In [16]:
for score, amount, act in ContributionAnalysis().annotated_top_processes(lca)[:10]:
        print([act['name'], act['unit'], act['location'], score / lca.score])

['Oil palm cultivation [FFB]', 'kilogram', 'MY', 0.9962398900508641]
['Soybean cultivation [soybean]', 'kilogram', 'BR', 0.002013211375567101]
['Forestry, logging and related service activities', 'ton', 'BR', 0.0005873861935007537]
['Forestry, logging and related service activities', 'ton', 'ID', 0.00032831793349268913]
['Cattle farming', 'ton', 'BR', 0.0001541748645263217]
['Forestry, logging and related service activities', 'ton', 'RU', 0.00012447406470098528]
['Cattle farming', 'ton', 'CN', 9.958745783993063e-05]
['Cultivation of oil seeds', 'ton', 'ID', 6.791705689847274e-05]
['Cultivation of crops nec', 'ton', 'ID', 5.674222416599837e-05]
['Cattle farming', 'ton', 'MX', 4.716075676088069e-05]
